# LFMC Estimation - Comparison Models
Runs the comparison tests
1. Train the out-of-site architecture using the within-site scenario
2. Train the within-site model architecture the out-of-site scenario
3. Train the Modis-tempCNN architecture using the within-site scenario
4. Train the Modis-tempCNN architecture using the out-of-site scenario

In [1]:
import os

import initialise
import common
from modelling_functions import create_models, run_experiment
from architecture_out_of_site import model_params
from model_parameters import ModelParams, ExperimentParams
import scenarios

## Directories and Input files
Change these settings as required
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `prism_csv`: The file containing extracted PRISM data for each sample, created by `Extract PRISM Data.ipynb`
- `aux_csv`: The file containing extracted sample labels, DEM, climate zone and other auxiliary data, created by `Extract Auxiliary Data.ipynb`.

In [2]:
modis_csv = os.path.join(common.DATASETS_DIR, 'modis_365days.csv')
prism_csv = os.path.join(common.DATASETS_DIR, 'prism_365days.csv')
aux_csv = os.path.join(common.DATASETS_DIR, 'samples_365days.csv')

## Set up experiment parameters
If the experiment dictionary contains a 'tests' key that is not 'falsy' (False, None, 0, empty list) it is assumed to be a list of tests to run. Each test will run with the specified model parameters. Model parameters not specified will be the same for each test, as set in the main model_params dictionary. A failed run can be restarted by setting the 'restart' key to the test that failed. This test and the remaining tests will then be run.

If 'tests' is 'falsy' then a single test will be run using the parameters in the main model_params dictionary.

Other settings are:
- layerTypes: specifies which layers to include in the model
- Layer parameters should be specified as a list. The first entry in the list will be used for the first layer, etc.
- If the experiment includes changes to the layers, all non-default layer parameters need to be included. The parameters that are kept constant can be specified by including a key for the layer type in the experiment dictionary, and the value set to a dictionary of the constant parameters.

Model_parameters that cannot be changed in tests are:
- \*Filename
- \*Channels
- targetColumn

Example of setting layer parameters:  
```
{'name': 'Filters',  
 'description': 'Test effect of different filter sizes on conv layers',  
 'tests': [{'conv': {'filters': [32, 32, 32]}},  
           {'conv': {'filters': [8, 8, 8]}},  
           {'conv': {'filters': [32, 8, 8]}},  
           {'conv': {'filters': [8, 32, 8]}},   
           {'conv': {'filters': [8, 8, 32]}},  
           {'conv': {'filters': [8, 16, 32]}},  
           {'conv': {'filters': [32, 16, 8]}}],  
 'conv': {'numLayers': 3, 'poolSize': [2, 3, 4]},  
 'restart': 0}
 ``` 

#### Note
As this experiment uses different architectures for each test, the notebook doesn't import the model architecture parameters. Instead, model architecture parameters are set in each test in the experiment parameters. 

In [3]:
experiment = ExperimentParams({
    'name': 'comparison_models_xxx',
    'description': 'Generate comparison models for scenarios using the "wrong" and the Modis-tempCNN architectures',
    'tests': [
# Out-of-site model within-site scenario     
        {'testName': 'Out-of-site model within-site scenario',
         'dataSources': ['optical', 'weather', 'aux'], 'batchSize': 512, 'dropoutRate': 0, 'epochs': 50,
         'auxColumns': ['Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos', 'Long_sin', 'Long_cos', 'Lat_norm'],
         'auxOneHotCols': ['Czone3'], 'auxAugment': True,
         'blocks': {
             'fc': [{'units': 128}],
             'opticalConv': [{'filters': 8, 'poolSize': 2},
                             {'filters': 8, 'poolSize': 3},
                             {'filters': 8, 'poolSize': 4}],
             'weatherConv': [{'filters': 8, 'poolSize': 2},
                             {'filters': 8, 'poolSize': 3},
                             {'filters': 8, 'poolSize': 4}]
             },
        },
# Within-site model out-of-site scenario
        {'testName': 'Within-site model out-of-site scenario',
         'dataSources': ['optical', 'weather', 'aux'], 'batchSize': 512, 'dropoutRate': 0.1, 'epochs': 100,
         'auxColumns': ['Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos', 'Long_sin', 'Long_cos', 'Lat_norm'],
         'auxOneHotCols': ['Czone3'], 'auxAugment': True,
         'blocks': {
             'fc': [{'units': 512}, {'units': 512}, {'units': 512}],
             'opticalConv': [{'filters': 8, 'poolSize': 0},
                             {'filters': 8, 'poolSize': 5},
                             {'filters': 8, 'poolSize': 2},
                             {'filters': 8, 'poolSize': 3},
                             {'filters': 8, 'poolSize': 4}],
             'weatherConv': [{'filters': 8, 'poolSize': 0},
                             {'filters': 8, 'poolSize': 5},
                             {'filters': 8, 'poolSize': 2},
                             {'filters': 8, 'poolSize': 3},
                             {'filters': 8, 'poolSize': 4}]
             },
        },
# Modis-tempCNN within-site scenario
        {'testName': 'Modis-tempCNN within-site scenario',
         'dataSources': ['optical', 'aux'], 'batchSize': 32, 'dropoutRate': 0.5, 'epochs': 100,
         'auxColumns': ['Day_sin', 'Day_cos', 'Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos', 'Long_sin', 'Long_cos', 'Lat_norm'], 
         'auxOneHotCols': [], 'auxAugment': False,
         'inputs': {'weather': None},
         'blocks': {
             'fc': [{'units': 256}, {'units': 256}],
             'opticalConv': [{'filters': 32, 'poolSize': 2},
                             {'filters': 32, 'poolSize': 3},
                             {'filters': 32, 'poolSize': 4}],
             'weatherConv': None,
             },
        }, 
# Modis-tempCNN out-of-site scenario
        {'testName': 'Modis-tempCNN out-of-site scenario',
         'dataSources': ['optical', 'aux'], 'batchSize': 32, 'dropoutRate': 0.5, 'epochs': 100,
         'auxColumns': ['Day_sin', 'Day_cos', 'Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos', 'Long_sin', 'Long_cos', 'Lat_norm'], 
         'auxOneHotCols': [], 'auxAugment': False,
         'inputs': {'weather': None},
         'blocks': {
             'fc': [{'units': 256}, {'units': 256}],
             'opticalConv': [{'filters': 32, 'poolSize': 2},
                             {'filters': 32, 'poolSize': 3},
                             {'filters': 32, 'poolSize': 4}],
             'weatherConv': None,
             },
        },
    ],

    'restart': None,
})

scenarios.within_site_scenario(experiment['tests'][0], set_common=False)
scenarios.out_of_site_scenario(experiment['tests'][1], set_common=False)
scenarios.within_site_scenario(experiment['tests'][2], set_common=False)
scenarios.out_of_site_scenario(experiment['tests'][3], set_common=False)

experiment

{'name': 'comparison_models_xxx',
 'description': 'Generate comparison models for scenarios using the "wrong" and the Modis-tempCNN architectures',
 'blocks': {},
 'tests': [{'testName': 'Out-of-site model within-site scenario',
   'dataSources': ['optical', 'weather', 'aux'],
   'batchSize': 512,
   'dropoutRate': 0,
   'epochs': 50,
   'auxColumns': ['Elevation',
    'Slope',
    'Aspect_sin',
    'Aspect_cos',
    'Long_sin',
    'Long_cos',
    'Lat_norm'],
   'auxOneHotCols': ['Czone3'],
   'auxAugment': True,
   'blocks': {'fc': [{'units': 128}],
    'opticalConv': [{'filters': 8, 'poolSize': 2},
     {'filters': 8, 'poolSize': 3},
     {'filters': 8, 'poolSize': 4}],
    'weatherConv': [{'filters': 8, 'poolSize': 2},
     {'filters': 8, 'poolSize': 3},
     {'filters': 8, 'poolSize': 4}]},
   'splitMethod': 'byYear',
   'yearColumn': 'Sampling year',
   'splitFolds': None,
   'splitYear': 2014,
   'yearFolds': 4},
  {'testName': 'Within-site model out-of-site scenario',
   'data

## Set up model parameters
Set up and customise the model parameters. To find out more about any parameter, run `model_params.help('<parameter>')` after running this cell to create the ModelParams object.

In [4]:
# Customize model parameters
scenarios.common_params(model_params)
model_params['modelName'] = experiment['name']
model_params['description'] = experiment['description']
model_params['samplesFile'] = aux_csv
model_params['modelRuns'] = common.EVALUATION_RUNS
model_params['modelDir'] = os.path.join(common.MODELS_DIR, model_params['modelName'])

model_params.add_input('optical', {'filename': modis_csv, 'channels': 7})
model_params.add_input('weather', {'filename': prism_csv, 'channels': 7})

# =============================================================================
# Parameters for parallel execution on GPUs
# =============================================================================
# model_params['gpuDevice'] = 1
# model_params['gpuMemory'] = 3800
# model_params['maxWorkers'] = 5
# model_params['enableXla'] = False
# model_params['mixedPrecision'] = None

model_params

{'modelName': 'comparison_models_xxx',
 'testName': None,
 'test': None,
 'run': None,
 'fold': None,
 'description': 'Generate comparison models for scenarios using the "wrong" and the Modis-tempCNN architectures',
 'modelClass': 'LfmcTempCnn',
 'modelDir': 'G:\\My Drive\\LFMC Data\\multi_modal_LFMC\\Models\\comparison_models_xxx',
 'tempDir': 'C:\\Temp\\LFMC',
 'diagnostics': False,
 'restartRun': None,
 'derivedModels': None,
 'saveModels': False,
 'saveFolds': True,
 'saveRunResults': False,
 'saveTrain': None,
 'saveValidation': False,
 'plotModel': True,
 'randomSeed': 1234,
 'modelSeed': 1234,
 'modelRuns': 20,
 'resplit': False,
 'seedList': [441,
  780,
  328,
  718,
  184,
  372,
  346,
  363,
  701,
  358,
  566,
  451,
  795,
  237,
  788,
  185,
  397,
  530,
  758,
  633,
  632,
  941,
  641,
  519,
  162,
  215,
  578,
  919,
  917,
  585,
  914,
  326,
  334,
  366,
  336,
  413,
  111,
  599,
  416,
  230,
  191,
  700,
  697,
  332,
  910,
  331,
  771,
  539,
  575,


## Build and run the models
Builds and trains the LFMC models. 

All models, predictions, and evaluation statisticsare saved to `model_dir`, with each test and run saved to a separate sub-directory. For each model created, predictions and evaluation statistics are also returned as attributes of the `model` object. These are stored as nested lists, the structure for a full experiment is:
- Tests (omitted if not an experiment)
  - Runs (omitted for a single run)
    - Folds (for k-fold splitting)

In [ ]:
models = run_experiment(experiment, model_params)
for model in models:
    display(getattr(model, 'test_stats', None))